## Import

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import plotly.plotly as py
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
cf.go_offline()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


## Load files

In [4]:
electric = pd.read_csv('Full Eletric Interval 042016.csv')

In [5]:
electric['Datetime'] = pd.to_datetime(electric['Date'] + ' ' + electric['Start Time'])

In [6]:
propertyCode = pd.read_excel('SIMS_Property_List_4-6-2016_Revised.xlsx')

In [7]:
weather = pd.read_csv('weather_prepared_final.csv')

In [8]:
weather.DATE = pd.to_datetime(weather.DATE)

In [9]:
searle = pd.read_csv('SearleModelData.csv')

In [10]:
jones = pd.read_csv('JonesModelData.csv')

In [11]:
reg = pd.read_csv('RegModelData.csv')

In [12]:
levi = pd.read_csv('LeviModelData.csv')

In [13]:
hinds = pd.read_csv('HindsModelData.csv')

In [14]:
physics = pd.read_csv('HighEnergyPhysicsModelData.csv')

In [15]:
GCIS = pd.read_csv('GCISModelData.csv')

In [16]:
accel = pd.read_csv('AcceleratorModelData.csv')

### List of codes

In [17]:
codelist = [b[0] for b in electric['Meter'].str.split()]

In [18]:
codes = pd.Series(codelist)

### Usage by day

In [19]:
electric_date_full = electric
electric_date_full.Datetime = electric_date_full.Datetime.dt.date

In [20]:
electric_day = electric_date_full.drop('Temperature', 1).groupby(by=['Meter', 'Datetime']).sum()

### Temperature

In [21]:
temperatures = weather.drop(['HUMIDITY', 'LON', 'LAT', 'STATION'], 1).iloc[1::4, :]

### Daily average temperature

In [22]:
daily_temperature = temperatures
daily_temperature.DATE = temperatures.DATE
daily_temperature = daily_temperature.groupby(by='DATE').mean()

### Humidity

In [23]:
humidity = weather.drop(['TEMPERATURE', 'LON', 'LAT', 'STATION'], 1).iloc[1::4, :]

### Daily average humidity

In [24]:
daily_humidity = humidity
daily_humidity.DATE = humidity.DATE
daily_humidity = daily_humidity.groupby(by='DATE').mean()

## Functions

In [193]:
def term(date):
    summer_14 = pd.date_range('2014-06-23', '2014-08-30')
    summer_fall_14 = pd.date_range('2014-08-31', '2014-09-28')
    fall_14 = pd.date_range('2014-09-29', '2014-12-13')
    winter_break_14 = pd.date_range('2014-12-14', '2015-01-04')
    winter_15 = pd.date_range('2015-01-05', '2015-03-21')
    spring_break_15 = pd.date_range('2015-03-22', '2015-03-29')
    spring_15 = pd.date_range('2015-03-30', '2015-06-13')
    spring_summer_15 = pd.date_range('2015-06-14', '2015-06-21')
    summer_15 = pd.date_range('2015-06-22', '2015-08-29')
    summer_fall_15 = pd.date_range('2015-08-30', '2015-09-27')
    fall_15 = pd.date_range('2015-09-28', '2015-12-12')
    winter_break_15 = pd.date_range('2015-12-13', '2016-01-03')
    winter_16 = pd.date_range('2016-01-04', '2016-03-19')
    spring_break_16 = pd.date_range('2016-03-20', '2016-03-27')
    spring_16 = pd.date_range('2016-03-28', '2016-06-11')
    spring_summer_16 = pd.date_range('2016-06-12', '2016-06-19')
    summer_16 = pd.date_range('2016-06-20', '2016-08-27')
    summer_fall_16 = pd.date_range('2016-08-28', '2016-09-25')
    fall_16 = pd.date_range('2016-09-26', '2016-12-10')
    winter_break_16 = pd.date_range('2016-12-11', '2016-01-02')
    winter_17 = pd.date_range('2017-01-03', '2017-03-18')
    spring_break_17 = pd.date_range('2017-03-19', '2017-03-26')
    spring_17 = pd.date_range('2017-03-27', '2017-06-10')
    spring_summer_17 = pd.date_range('2017-06-11', '2017-06-18')
    summer_17 = pd.date_range('2017-06-19', '2017-08-26')
    summer_fall_17 = pd.date_range('2017-08-27', '2017-09-24')
    fall_17 = pd.date_range('2017-09-25', '2017-12-09')
    winter_break_17 = pd.date_range('2017-12-10', '2017-01-02')
    winter_18 = pd.date_range('2017-01-03', '2017-03-17')
    spring_break_18 = pd.date_range('2017-03-18', '2017-03-25')
    spring_18 = pd.date_range('2017-03-26', '2017-06-09')
    
    # Summer
    if date in summer_14 | summer_15 | summer_16 | summer_17:
        return summer_14.append(summer_15).append(summer_16).append(summer_17)
    # Fall
    elif date in fall_14 | fall_15 | fall_16 | fall_17:
        return fall_14.append(fall_15).append(fall_16).append(fall_17)
    # Winter
    elif date in winter_15 | winter_16 | winter_17 | winter_18:
        return winter_15.append(winter_16).append(winter_17).append(winter_18)
    # Spring
    elif date in spring_16 | spring_15 | spring_17 | spring_18:
        return spring_16.append(spring_15).append(spring_17).append(spring_18)
    # Summer-Fall break
    elif date in summer_fall_14 | summer_fall_15 | summer_fall_16 | summer_fall_17:
        return summer_fall_14.append(summer_fall_15).append(summer_fall_16).append(summer_fall_17)
    # Winter break
    elif date in winter_break_14 | winter_break_15 | winter_break_16 | winter_break_17:
        return winter_break_14.append(winter_break_15).append(winter_break_16).append(winter_break_17)
    # Spring break
    elif date in spring_break_15 | spring_break_16 | spring_break_17 | spring_break_18:
        return spring_break_15.append(spring_break_16).append(spring_break_16).append(spring_breaK_18)
    # Spring-Summer break
    elif date in spring_summer_15 | spring_summer_16 | spring_summer_17:
        return spring_summer_15.append(spring_summer_16).append(spring_summer_17)

In [26]:
def normal_usage(meter, date, temperature):
    is_weekend = pd.to_datetime(date).weekday() in [5,6]
    meter_usage = electric_day.ix[meter]
    term_same = pd.to_datetime(meter_usage.index).isin(term(date))
    weekend_same = pd.to_datetime(meter_usage.index).weekday.isin([5,6]) == is_weekend
    all_temperatures = daily_temperature.ix[meter_usage.index]['TEMPERATURE']
    temperature_greater = all_temperatures > temperature - 5.5
    temperature_less = all_temperatures < temperature + 5.5
    temperature_near = temperature_greater & temperature_less
    #all_humidities = daily_humidity.ix[meter_usage.index]['HUMIDITY']
    #humidity_greater = all_humidities > humidity - 12
    #humidity_less = all_humidities < humidity + 12
    #humidity_near = humidity_greater & humidity_less
    day_different = np.invert(pd.to_datetime(meter_usage.index).isin([date]))
    number_averaged = meter_usage[term_same & temperature_near & weekend_same & day_different]['Usage'].count()
    return (meter_usage[term_same & temperature_near & weekend_same & day_different]['Usage'].mean(), number_averaged)

In [27]:
def percent_difference(meter, date, temperature, usage):
    normal = normal_usage(meter, date, temperature)
    normal_use = normal[0]
    normal_number_averaged = normal[1]
    if usage > 100:
        if normal_use == 0:
            normal_use = 1
        return (100 * (usage - normal_use) / normal_use, normal_number_averaged)
    return (float('nan'), float('nan'))

In [28]:
def percent_diff_lst(meter):
    dates = pd.to_datetime(electric_day.ix[meter].index)
    start = pd.to_datetime('2015-01-01')
    end = pd.to_datetime('2016-04-10')
    electric_day_temp = electric_day.ix[meter][(dates >= start) & (dates <= end)]
    diff_lst = []
    date_lst = []
    num_lst = []
    for day in electric_day_temp.index:
        temperature = float(daily_temperature.ix[pd.to_datetime(day).date()])
        #humidity = float(daily_humidity.ix[pd.to_datetime(day).date()])
        usage = float(electric_day_temp.ix[pd.to_datetime(day).date()])
        diff_tuple = percent_difference(meter, day.strftime('%Y-%m-%d'), temperature, usage)
        diff = diff_tuple[0]
        num = diff_tuple[1]
        diff_lst.append(diff)
        date_lst.append(day)
        num_lst.append(num)
    return pd.DataFrame({'Percent Difference' : diff_lst, 'Number Averaged' : num_lst}, date_lst)

In [29]:
def anomalies(meter):
    diff = percent_diff_lst(meter)
    return diff[diff['Percent Difference'] > 12]

In [30]:
def normal_usage_lst(meter):
    dates = pd.to_datetime(electric_day.ix[meter].index)
    start = pd.to_datetime('2015-01-01')
    end = pd.to_datetime('2016-04-10')
    electric_day_temp = electric_day.ix[meter][(dates >= start) & (dates <= end)]
    norm_lst = []
    date_lst = []
    num_lst = []
    for day in electric_day_temp.index:
        temperature = daily_temperature.ix[pd.to_datetime(day).date()]
        #humidity = daily_humidity.ix[pd.to_datetime(day).date()]
        usage = float(electric_day_temp.ix[pd.to_datetime(day).date()])
        norm_tuple = normal_usage(meter, day.strftime('%Y-%m-%d'), float(temperature))
        norm = norm_tuple[0]
        num = norm_tuple[1]
        norm_lst.append(norm)
        date_lst.append(day)
        num_lst.append(num)
    return pd.DataFrame({'Normal Usage' : norm_lst, 'Number Averaged' : num_lst}, date_lst)

In [31]:
def compare_normal_usage(meter):
    normal = normal_usage_lst(meter)
    actual = electric_day.ix[meter]
    fig = {
        'data': [
            {'x': normal.index, 'y': normal['Normal Usage'], 'mode': 'markers', 'name': 'Average Usage'},
            {'x': actual.index, 'y': actual.Usage, 'mode': 'markers', 'name': 'Actual Usage'}
        ],
        'layout': {
            'xaxis': {'title': 'Date'},
            'yaxis': {'title': 'Usage'}
        }
    }
    iplot(fig)

## Examples

In [31]:
percent_diff_lst('E44 Booth Harper Center (B4)').iplot(mode='markers', x='Number Averaged', y='Percent Difference')

/usr/local/conda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix

/usr/local/conda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix

/usr/local/conda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



In [32]:
normal_usage_lst('E44 Booth Harper Center (B4)')['Number Averaged'].mean()

/usr/local/conda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



15.515021459227468

In [33]:
normal_usage_lst('E44 Booth Harper Center (B4)')['Number Averaged'].median()

/usr/local/conda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



13.0

In [118]:
percent_diff_lst('E44 Booth Harper Center (B4)').iplot(mode='markers', y='Number Averaged')

/usr/local/conda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



In [103]:
anomalies('E44 Booth Harper Center (B4)')

/usr/local/conda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



,Number Averaged,Percent Difference
2015-03-13,20,39.443371
2015-03-15,13,142.186519
2015-03-16,7,48.692015
2015-04-01,39,33.471291
2015-04-05,17,16.390949
2015-04-09,39,67.477326
2015-05-08,19,21.709185
2015-05-11,37,52.938883
2015-05-14,44,62.328833
2015-05-16,8,25.859589


In [119]:
compare_normal_usage('B36 BSLC (B1)')

/usr/local/conda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



In [159]:
electric.Meter.unique()

array(['A06 Crerar Library (B1)', 'F02 Steam Plant (B1)',
       'G02 Capin Hall (B1)', 'D36 Social Science Building (B1)',
       'H01 Computers 1155 Building (B1)', 'A11 Kovler Laboratories (B1)',
       'A08 Hinds Laboratories (R1)', 'A84 American School (B1)',
       'A62 Cummings Life Sciences (B1)', 'A13 Bookstore (B1)',
       'H02 Law School (B1)', 'A22 Carlson Animal Research (B4)',
       'F02 Steam Plant (B2)', 'C02 Crown Field House (B1)',
       'E21 Breckinridge House (B1)', 'D14 Center Gender Studies (B1)',
       'L07 5482 S. Greenwood Ave (R1)', 'D25 Walker Museum (R1)',
       'D25 Walker Museum (R2)', 'I13 Woodlawn Social Service (B1)',
       'D53 Pick Hall (R1)', 'N11 ATS (B1)',
       'B34 Stagg Field Building (B1)', 'D16 Eckhart/Ryerson (B1)',
       'D16 Eckhart/Ryerson (B2)', 'C31 54th & Ellis Parking (B1)',
       'C31 54th & Ellis Parking (B2)', 'D20 Levi Hall (B1)',
       'I10 6022-24 S. Drexel (B3)', 'B36 BSLC (B1)',
       'A50 MRS Building (CLSC) (B1)', 

In [76]:
electric_day.ix['N11 ATS (U1)'].join(daily_temperature).iplot(x='TEMPERATURE', y='Usage', mode='markers')

In [77]:
electric_day.ix['N11 ATS (U1)'].join(daily_humidity).iplot(x='HUMIDITY', y='Usage', mode='markers')

In [34]:
searle

,DateTime,SearlekWh,Temperature
0,06-14-2016 00:00:00,517.32,72.0
1,06-14-2016 01:00:00,517.41,72.0
2,06-14-2016 02:00:00,515.76,72.0
3,06-14-2016 03:00:00,513.21,72.0
4,06-14-2016 04:00:00,513.03,71.0
5,06-14-2016 05:00:00,532.41,70.0
6,06-14-2016 06:00:00,531.36,71.0
7,06-14-2016 07:00:00,535.05,73.0
8,06-14-2016 08:00:00,539.07,75.0
9,06-14-2016 09:00:00,542.88,79.0


## Combine old Regenstein data

In [44]:
old_reg = electric[codes.isin(['C03'])]
old_reg['Datetime'] = pd.to_datetime(old_reg['Date'] + ' ' + old_reg['Start Time'])
old_reg = old_reg.groupby(by='Datetime').sum()

/usr/local/conda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [56]:
old_reg.reset_index()

,Datetime,Usage,Temperature
0,2014-04-11 23:30:00,425.73025,NaN
1,2014-04-12 00:00:00,646.45055,104.0
2,2014-04-12 00:30:00,518.22145,104.0
3,2014-04-12 01:00:00,505.37375,110.0
4,2014-04-12 01:30:00,661.79230,110.0
5,2014-04-12 02:00:00,417.87070,114.0
6,2014-04-12 02:30:00,625.36610,114.0
7,2014-04-12 03:00:00,511.80480,122.0
8,2014-04-12 03:30:00,490.02335,122.0
9,2014-04-12 04:00:00,634.64255,134.0


In [87]:
def normal_usage_reg(date, temperature):
    is_weekend = pd.to_datetime(date).weekday() in [5,6]
    meter_usage = old_reg
    term_same = pd.to_datetime(meter_usage.index).isin(term(date))
    weekend_same = pd.to_datetime(meter_usage.index).weekday.isin([5,6]) == is_weekend
    all_temperatures = daily_temperature.ix[meter_usage.index]['TEMPERATURE']
    temperature_greater = all_temperatures > temperature - 5.5
    temperature_less = all_temperatures < temperature + 5.5
    temperature_near = temperature_greater & temperature_less
    #all_humidities = daily_humidity.ix[meter_usage.index]['HUMIDITY']
    #humidity_greater = all_humidities > humidity - 12
    #humidity_less = all_humidities < humidity + 12
    #humidity_near = humidity_greater & humidity_less
    day_different = np.invert(pd.to_datetime(meter_usage.index).isin([date]))
    number_averaged = meter_usage[term_same & temperature_near & weekend_same & day_different]['Usage'].count()
    return (meter_usage[term_same & temperature_near & weekend_same & day_different]['Usage'].mean(), number_averaged)

In [91]:
def percent_difference_reg(date, temperature, usage):
    normal = normal_usage_reg(date, temperature)
    normal_use = normal[0]
    normal_number_averaged = normal[1]
    if usage > 100:
        if normal_use == 0:
            normal_use = 1
        return (100 * (usage - normal_use) / normal_use, normal_number_averaged)
    return (float('nan'), float('nan'))